In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
import sys
from pathlib import Path

parent = str(Path().absolute().parent)
if parent not in sys.path:
    sys.path.insert(0,parent)
from sherlock.evidence_store import EvidenceStore
from sherlock.models import Claim, Evidence, Argument, EvidenceCollection
from sherlock.agents import ClaimInvestgiationAgent
from sherlock.utils import export_argdown
import pyperclip

In [ ]:
store = EvidenceStore("wally_evidence")

# Add our evidence
evidence_ids = []
evidence_ids.append(store.add_evidence("Librarian Jane saw someone matching Wally's description at 3pm"))
evidence_ids.append(store.add_evidence("Security camera shows someone in red and white stripes in the park"))
evidence_ids.append(store.add_evidence("Library card records don't show Wally checking out any books today"))
evidence_ids.append(store.add_evidence("Wally's phone GPS shows he was at the library coordinates"))



In [ ]:


agent_pro = ClaimInvestgiationAgent(store, supports=True)
agent_con = ClaimInvestgiationAgent(store, supports=False)
# Create a test claim
wally_claim = Claim(text="Wally is in the library")


In [ ]:
wally_claim = agent_pro.evaluate_claim(wally_claim)
wally_claim = agent_con.evaluate_claim(wally_claim)
# Evaluate it
wally_claim.likelihood

In [ ]:
arg = export_argdown(wally_claim)

In [ ]:
with open('argdown.txt', 'w') as f:
    f.write(arg)